In [1]:
import os
import sys

import numpy as np
#sys.path.insert(1, r"C:\Users\Zack\OneDrive\BCI\arl-eegmodels-master")
#from EEGModels import ShallowConvNet

# main directory
main_dir = os.getcwd()
# eeg data directory
eeg_dir = os.path.join(main_dir, "eeg_data")
# eeg validation data directory
eeg_validation_dir = os.path.join(main_dir, "eeg_data_validation")

MI_CLASSES = ["left", "right", "none"]

In [2]:
def to_one_sec(eeg_data, num_sec=5, samp_freq=250):
    one_sec_data = []
    for i in range(num_sec):
        eeg_data_i = eeg_data[i*samp_freq:(i+1)*samp_freq]
        one_sec_data.append(eeg_data_i)

    return one_sec_data

In [17]:
def create_data(data_dir):
    training_data_dict = {}

    for mi_class in MI_CLASSES:
        if mi_class not in training_data_dict:
            training_data_dict[mi_class] = []

        mi_class_dir = os.path.join(data_dir, mi_class)
        for filename in os.listdir(mi_class_dir):
            if "filtered" in filename:
                eeg_data = np.loadtxt(os.path.join(mi_class_dir, filename), delimiter=',')
                one_sec_list = to_one_sec(eeg_data, num_sec=5, samp_freq=250)
                for i in range(len(one_sec_list)):  # for each 1 second trial
                    one_sec_data = one_sec_list[i] # (250, 8) eeg data
                    one_sec_dataT = np.transpose(one_sec_data)  # (8, 250) eeg data
                    training_data_dict[mi_class].append(one_sec_dataT)  

    print("Trials per class: " + str(int(len(os.listdir(mi_class_dir))/3)) + "\n")
    session_count = [len(training_data_dict[mi_class]) for mi_class in MI_CLASSES]
    print(f"One sec trial count:\nLeft: {session_count[0]}, Right: {session_count[1]}, None: {session_count[2]}\n")

    for mi_class in MI_CLASSES:
        np.random.shuffle(training_data_dict[mi_class])  # randomize session order
        training_data_dict[mi_class] = training_data_dict[mi_class][:min(session_count)]  # use min session count number of sessions

    # creating X, y 
    labeled_data = []
    for mi_class in MI_CLASSES:
        for data in training_data_dict[mi_class]:
            if mi_class == "left":
                labeled_data.append([data, [1, 0, 0]])
            elif mi_class == "right":
                labeled_data.append([data, [0, 1, 0]])
            elif mi_class == "none":
                labeled_data.append([data, [0, 0, 1]])

    np.random.shuffle(labeled_data)
    
    return labeled_data


In [18]:
labeled_data = create_data(data_dir=eeg_dir)

Trials per class: 15

One sec trial count:
Left: 75, Right: 75, None: 75



In [38]:
labeled_data = create_data(data_dir=eeg_dir)

print("Creating labeled data...")
labeled_data_X = []
labeled_data_y = []

for X, y in labeled_data:
    labeled_data_X.append(X)
    labeled_data_y.append(y)
print("labeled_data_X: " + str(np.shape(labeled_data_X)))
print("labeled_data_y: " + str(np.shape(labeled_data_y)) + "\n")

print("Creating testing data...")
train_X = labeled_data_X[0:9]
train_y = labeled_data_y[0:9]
print("labeled_data_X: " + str(np.shape(train_X)))
print("labeled_data_y: " + str(np.shape(train_y)) + "\n")

print("Creating training data...")
test_X = labeled_data_X[9:12]
test_y = labeled_data_y[9:12]
print("labeled_data_X: " + str(np.shape(test_X)))
print("labeled_data_y: " + str(np.shape(test_y)) + "\n")

print("Creating validation data...")
validation_X = labeled_data_X[12:]
validation_y = labeled_data_y[12:]
print("labeled_data_X: " + str(np.shape(validation_X)))
print("labeled_data_y: " + str(np.shape(validation_y)) + "\n")

Trials per class: 15

One sec trial count:
Left: 75, Right: 75, None: 75

Creating labeled data...
labeled_data_X: (225, 8, 250)
labeled_data_y: (225, 3)

Creating testing data...
labeled_data_X: (9, 8, 250)
labeled_data_y: (9, 3)

Creating training data...
labeled_data_X: (3, 8, 250)
labeled_data_y: (3, 3)

Creating validation data...
labeled_data_X: (213, 8, 250)
labeled_data_y: (213, 3)



In [30]:
np.shape(labeled_data_X[2])

(8, 250)

In [6]:
#train_X = np.transpose(np.array(train_X))
train_X = np.array(train_X)
print("train_X shape: " + str(np.shape(train_X)))
#test_X = np.transpose(np.array(test_X))
test_X = np.array(test_X)
print("test_X shape: " + str(np.shape(test_X)))

#train_y = np.transpose(np.array(train_y))
train_y = np.array(train_y)
print("train_y shape: " + str(np.shape(train_y)))
#test_y = np.transpose(np.array(test_y))
test_y = np.array(test_y)
print("test_y shape: " + str(np.shape(test_y)))

train_X shape: (60, 8, 250)
test_X shape: (15, 8, 250)
train_y shape: (60, 3)
test_y shape: (15, 3)


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, AveragePooling2D
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, BatchNormalization
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [8]:
    def square(x):
        return K.square(x)

    def log(x):
        return K.log(K.clip(x, min_value = 1e-7, max_value = 10000))   

    def ShallowConvNet(nb_classes, Chans = 8, Samples = 250, dropoutRate = 0.5):
        # start the model
        input_main   = Input((Chans, Samples, 1))
        block1       = Conv2D(40, (1, 25), 
                                        input_shape=(Chans, Samples, 1),
                                        kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
        block1       = Conv2D(40, (Chans, 1), use_bias=False, 
                                kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
        block1       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block1)
        block1       = Activation(square)(block1)
        block1       = AveragePooling2D(pool_size=(1, 75), strides=(1, 15))(block1)
        block1       = Activation(log)(block1)
        block1       = Dropout(dropoutRate)(block1)
        flatten      = Flatten()(block1)
        dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
        softmax      = Activation('softmax')(dense)

        return Model(inputs=input_main, outputs=softmax)

In [9]:
model = ShallowConvNet(nb_classes=3, Chans=8, Samples=250)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8, 250, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 8, 226, 40)        1040      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 226, 40)        12800     
_________________________________________________________________
batch_normalization (BatchNo (None, 1, 226, 40)        160       
_________________________________________________________________
activation (Activation)      (None, 1, 226, 40)        0         
_________________________________________________________________
average_pooling2d (AveragePo (None, 1, 11, 40)         0         
_________________________________________________________________
activation_1 (Activation)    (None, 1, 11, 40)         0     

In [10]:
model.compile(loss="categorical_crossentropy", optimizer ="adam", metrics=["accuracy"])

In [11]:
epochs = 50
batch_size = 10
fitted_model = model.fit(train_X, train_y, batch_size=batch_size, epochs=50, validation_data=(test_X, test_y))
#fitted_model = model.fit(train_X, train_y, batch_size=batch_size, epochs=epochs, verbose=2)


Epoch 1/50
6/6 [==============================] - 1s 49ms/step - loss: 1.4404 - accuracy: 0.5388 - val_loss: 1.4870 - val_accuracy: 0.3333
Epoch 2/50
6/6 [==============================] - 0s 15ms/step - loss: 0.9836 - accuracy: 0.5824 - val_loss: 1.0343 - val_accuracy: 0.3333
Epoch 3/50
6/6 [==============================] - 0s 13ms/step - loss: 0.6929 - accuracy: 0.6979 - val_loss: 0.8666 - val_accuracy: 0.6667
Epoch 4/50
6/6 [==============================] - 0s 18ms/step - loss: 0.3960 - accuracy: 0.8433 - val_loss: 1.8795 - val_accuracy: 0.6667
Epoch 5/50
6/6 [==============================] - 0s 16ms/step - loss: 0.4176 - accuracy: 0.8574 - val_loss: 2.8428 - val_accuracy: 0.6667
Epoch 6/50
6/6 [==============================] - 0s 14ms/step - loss: 0.5414 - accuracy: 0.8207 - val_loss: 2.4045 - val_accuracy: 0.3333
Epoch 7/50
6/6 [==============================] - 0s 13ms/step - loss: 0.6002 - accuracy: 0.7979 - val_loss: 2.6333 - val_accuracy: 0.0000e+00
Epoch 8/50
6/6 [=======

In [75]:
model = Sequential()

model.add(Conv1D(64, (3), input_shape=np.shape(train_X)))
model.add(Activation('relu'))

model.add(Conv1D(64, (2)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=(2)))

model.add(Conv1D(64, (2)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=(2)))

model.add(Flatten())

model.add(Dense(512))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

ValueError: Input 0 of layer max_pooling1d_2 is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, 60, 5, 64)